In [37]:
import pandas as pd
import folium
from folium.plugins import MiniMap
from pathlib import Path

# === 1. Baca data dari Google Sheets ===
url = "https://docs.google.com/spreadsheets/d/1iNevuUEhHf16MPlA-WDD1vDxiTRc5A4hGeBHZjh4XPk/export?format=csv"
data = pd.read_csv(url)

# Rapikan nama kolom
data.columns = data.columns.str.strip().str.upper()

# Pastikan kolom JENIS berisi teks
data["JENIS"] = data["JENIS"].astype(str).str.strip().fillna("Lainnya")

# === 2. Tentukan lokasi awal peta ===
center_lat = data["LINTANG"].mean()
center_lon = data["BUJUR"].mean()

# === 3. Buat Peta Utama ===
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=6,
    tiles="OpenStreetMap",
    zoom_control=False   # ini yang mematikan tombol zoom
)

# Tambah basemap lain
folium.TileLayer("cartodb positron", name="Carto Light").add_to(m)
folium.TileLayer("cartodb dark_matter", name="Carto Dark").add_to(m)
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    attr="Tiles © Esri",
    name="Esri Satellite",
    overlay=False,
    control=True
).add_to(m)

# === 10. Tambahkan layer Kawasan Konservasi ===
import json

# Baca file GeoJSON
geojson_path = Path("Kawasan-Konservasi.json")

with open(geojson_path, encoding="utf-8") as f:
    kawasan_data = json.load(f)

# Tambahkan ke peta tanpa masuk ke filter
folium.GeoJson(
    kawasan_data,
    name="Kawasan Konservasi",
    style_function=lambda feature: {
        'fillColor': '#FFFF00',
        'color': '#FFFF00',
        'weight': 0,
        'fillOpacity': 0.5,
    }
).add_to(m)

# === 4. Warna fallback kalau ikon tidak ditemukan ===
warna_dict = {
    "Penyu": "green",
    "Duyung": "blue",
    "Kima": "purple",
    "Lumba-lumba": "orange",
    "Lainnya": "red"
}

# === 5. Tambahkan Checkbox "Centang Semua" di LayerControl ===
checkall_js = """
<script>
document.addEventListener('DOMContentLoaded', function() {
    var interval = setInterval(function() {
        var control = document.querySelector('.leaflet-control-layers-overlays');
        if (control) {
            clearInterval(interval);
            var wrapper = document.createElement('div');
            wrapper.innerHTML = `
                <label style="font-weight:600; display:flex; align-items:center; margin-bottom:4px;">
                    <input type="checkbox" id="toggle-all" style="margin-right:6px;">
                    Filter / Hapus Filter
                </label>
            `;
            control.insertBefore(wrapper, control.firstChild);
            var toggleAll = document.getElementById('toggle-all');
            toggleAll.addEventListener('change', function() {
                var boxes = control.querySelectorAll('input.leaflet-control-layers-selector[type="checkbox"]');
                boxes.forEach(function(box) {
                    if (box.checked !== toggleAll.checked) box.click();
                });
            });
        }
    }, 300);
});
</script>
"""
m.get_root().html.add_child(folium.Element(checkall_js))

# === 6. Tambahkan Marker per Jenis ===
for jenis in data["JENIS"].unique():
    group = folium.FeatureGroup(name=f"{jenis}", show=True).add_to(m)
    subset = data[data["JENIS"] == jenis]

    color = warna_dict.get(jenis, "gray")

    for _, row in subset.iterrows():
        popup_text = f"""
        <b>KELOMPOK:</b> {row.get('KELOMPOK','')}<br>
        <b>SPESIES:</b> {row.get('SPESIES','')}<br>
        <b>NAMA ID:</b> {row.get('NAMA_ID','')}<br>
        <b>JUMLAH TEMU:</b> {row.get('JUMLAH_TEMU','')}<br>
        <b>DESKRIPSI:</b> {row.get('DESKRIPSI','')}<br>
        <b>TAHUN:</b> {row.get('TAHUN','')}<br>
        <b>LINK LAPOR:</b> <a href="{row.get('LINK_LAPOR','#')}" target="_blank">Klik di sini</a>
        """

        icon_filename = f"icon{jenis.lower().replace(' ', '')}.png"
        icon_path = Path(icon_filename)
        if icon_path.exists():
            icon = folium.CustomIcon(icon_image=str(icon_path), icon_size=(40, 40))
        else:
            icon = folium.Icon(color=color, icon="info-sign")

        folium.Marker(
            location=[row["LINTANG"], row["BUJUR"]],
            popup=folium.Popup(popup_text, max_width=500),
            tooltip=row.get("SPESIES", ""),
            icon=icon
        ).add_to(group)

# === 7. Tambahkan Layer Control ===
folium.LayerControl(collapsed=False).add_to(m)

# === 8. Judul Sticky di kiri atas + Logo di atas judul (Responsif, tetap sejajar di desktop) ===
title_html = '''
     <div style="
         position: fixed;
         top: 15px;
         left: 20px;
         background-color: rgba(255, 255, 255, 0.95);
         padding: 25px 28px 20px 28px;
         border-radius: 14px;
         font-family: 'Segoe UI', Arial, sans-serif;
         box-shadow: 0 4px 12px rgba(0,0,0,0.25);
         z-index: 9999;
         max-width: 380px;
         backdrop-filter: blur(6px);
         text-align: center;
     ">
         <!-- Logo di atas (sejajar di desktop) -->
         <div class="logo-container" style="
             display: flex;
             justify-content: center;
             align-items: center;
             gap: 20px;
             margin-bottom: 15px;
             flex-direction: row;
         ">
             <img src="LogoKKP.png" alt="Logo KKP" class="logo-item" height="40" style="object-fit: contain;">
             <img src="LogoBKKPN.png" alt="Logo BKKPN Kupang" class="logo-item" height="40" style="object-fit: contain;">
         </div>

         <!-- Judul -->
         <div style="font-size: 22px; font-weight: 800; color: #0b3d91; margin-bottom: 10px;">
             🌊 PETA SEBARAN BIOTA LAUT DILINDUNGI
         </div>

         <!-- Subjudul -->
         <div style="font-size: 14px; font-weight: 600; color: #004080; margin-bottom: 10px;">
             Balai Kawasan Konservasi Perairan Nasional Kupang
         </div>

         <!-- Deskripsi -->
         <div style="font-size: 16px; color: #2c3e50; line-height: 1.5;">
             Klik tombol di bawah untuk melihat grafik monitoring dan dashboard:
         </div>

         <!-- Tombol -->
         <div style="margin-top: 12px; display: flex; flex-direction: column; gap: 8px;">
             <a href="https://lookerstudio.google.com/u/0/reporting/faefd501-627c-47bf-b741-b25ebb73af8c/page/p_tu61f29uwd" 
                target="_blank" 
                style="
                    background-color: #007BFF;
                    color: white;
                    text-align: center;
                    padding: 10px;
                    border-radius: 8px;
                    font-weight: 600;
                    text-decoration: none;
                    font-size: 16px;
                    transition: 0.3s;
                "
                onmouseover="this.style.backgroundColor='#0056b3'"
                onmouseout="this.style.backgroundColor='#007BFF'">
                📊 Lihat Grafik Monitoring
             </a>
             <a href="https://sites.google.com/view/biota-laut-dilindungi/home" 
                target="_blank" 
                style="
                    background-color: #28a745;
                    color: white;
                    text-align: center;
                    padding: 10px;
                    border-radius: 8px;
                    font-weight: 600;
                    text-decoration: none;
                    font-size: 16px;
                    transition: 0.3s;
                "
                onmouseover="this.style.backgroundColor='#1e7e34'"
                onmouseout="this.style.backgroundColor='#28a745'">
                🌐 Buka Dashboard SI MOLI
             </a>
         </div>
     </div>

     <!-- CSS Responsif -->
     <style>
         @media (max-width: 768px) {
             .logo-container {
                 flex-direction: column !important;
                 gap: 8px !important;
             }
             .logo-item {
                 height: 45px !important;
             }
             div[style*="max-width: 380px;"] {
                 max-width: 90% !important;
                 left: 50% !important;
                 transform: translateX(-50%) !important;
                 padding: 18px 16px !important;
             }
         }
     </style>
'''
m.get_root().html.add_child(folium.Element(title_html))

# === 9. Legenda di kiri bawah (lebih lebar & rapi) ===
legend_html = '''
     <div style="
         position: fixed; 
         bottom: 25px; left: 25px; 
         width: 260px; 
         background-color: rgba(255, 255, 255, 0.9);
         border-radius: 10px;
         padding: 14px 18px;
         box-shadow: 0 3px 8px rgba(0,0,0,0.25);
         font-family: 'Segoe UI', Arial, sans-serif;
         font-size: 13px;
         z-index: 9999;
     ">
         <b style="font-size:15px;">🗺️ Legenda</b>
         <div style="margin-left:8px; line-height:1.8; margin-top:5px;">
             <div><img src="iconpenyu.png" width="26" style="vertical-align:middle; margin-right:8px;"> Penyu</div>
             <div><img src="icondugong.png" width="26" style="vertical-align:middle; margin-right:8px;"> Dugong</div>
             <div><img src="iconlumba-lumba.png" width="26" style="vertical-align:middle; margin-right:8px;"> Lumba-lumba</div>
             <div><img src="iconpaus.png" width="26" style="vertical-align:middle; margin-right:8px;"> Paus</div>
             <div><img src="iconhiu.png" width="26" style="vertical-align:middle; margin-right:8px;"> Hiu</div>
             <div><img src="iconhiupaus.png" width="26" style="vertical-align:middle; margin-right:8px;"> Hiu Paus</div>
             <div><img src="iconmanta.png" width="26" style="vertical-align:middle; margin-right:8px;"> Manta</div>
             <div><img src="iconkima.png" width="26" style="vertical-align:middle; margin-right:8px;"> Kima</div>
         </div>
     </div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# === 10. Simpan ke file HTML ===
Path("index.html").write_text(m.get_root().render(), encoding="utf-8")
print("✅ Peta berhasil dibuat → buka file: index.html")


✅ Peta berhasil dibuat → buka file: index.html
